In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [7]:
library(googlesheets)
gs_token <- "/home/users/ytanigaw/.googlesheets_token.rds"
gs_auth(token = gs_token)


Auto-refreshing stale OAuth token.


In [29]:
# input
phe_info_f <- '../../common/biomarker.phenotype.info.tsv'
eval_f <- 'snpnet_prs_eval.tsv'
ST_sheet <- 'https://docs.google.com/spreadsheets/d/1j8q1Y7wnMg9nWUm0iT4wJvFfg_hgIXbrtvxelqWHeH4' 

# output
out_f <- 'combined_table.tsv'


In [30]:
phe_info_df <- fread(phe_info_f) %>% rename('Phenotype'='name')

In [5]:
eval_df <- fread(eval_f) %>%
rename('trait'='#trait')

In [8]:
current_tbl_df <- ST_sheet %>% gs_url() %>% 
gs_read(ws = 'UKBB PRS prediction')


Sheet-identifying info appears to be a browser URL.
googlesheets will attempt to extract sheet key from the URL.
Putative key: 1j8q1Y7wnMg9nWUm0iT4wJvFfg_hgIXbrtvxelqWHeH4
Warning message:
“`data_frame()` is deprecated as of tibble 1.1.0.
Please use `tibble()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”Sheet successfully identified: "UKBB Biomarker Paper Supplementary Tables Revisions"
Warning message:
“`as_data_frame()` is deprecated as of tibble 2.0.0.
Please use `as_tibble()` instead.
The signature and semantics have changed, see `?as_tibble`.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”Accessing worksheet titled 'UKBB PRS prediction'.
Parsed with column specification:
cols(
  Trait = col_character(),
  Population = col_character(),
  `Spearman correlation` = col_double(),
  `Number of individuals` = col_double(),
  `P-va

In [33]:
name_dict <- setNames(    
    phe_info_df$Phenotype,
    phe_info_df$annotation    
)


In [35]:
combined_df <- current_tbl_df %>%
inner_join(
    eval_df %>%
    mutate(pop_split = str_replace(pop_split, 'white_british:test', 'white_british')) %>%
    filter(!str_detect(pop_split, ':')),
    by=c('Trait'='trait', 'Population'='pop_split')
) %>%
left_join(
    phe_info_df %>% select(Phenotype, annotation),
    by=c('Trait'='annotation')
) %>%
mutate(Trait = Phenotype) %>%
select(-Phenotype) %>%
arrange(Population, Trait)


In [37]:
combined_df %>%
fwrite(out_f, sep='\t', na = "NA", quote=F)
